In [ ]:
# SPDX-License-Identifier: BSD-3-Clause
# Copyright 2022-2023, 2025 NXP

# Models download
Purpose of this notebook is to download model sources and metadata used by the examples, and apply necessary conversion steps prior to execution. Resulting models will be located in the top level `downloads` directory.

This notebook has dependency on availability of eIQ Toolkit environment being configured.
Refer to [downloads\README.md](./README.md) for instructions.

## Preamble


In [ ]:
import os
from pathlib import Path
import re
import shutil
import subprocess
import tarfile
import urllib.parse
import urllib.request
import zipfile
import tensorflow as tf
import numpy as np

cwd = Path.cwd()

# Repository directory
top_dir = str(cwd.parent.absolute())
# Models directory
models_dir = os.path.join(top_dir, 'downloads', 'models')
# Media directory
media_dir = os.path.join(top_dir, 'downloads', 'media')
# Work directory (temporary)
tmp_dir = os.path.join(top_dir, 'tmp')
# Small batch of pictures samples for post-training quantization
samples_dir = os.path.join(top_dir, 'samples')
# Neutron converter binary
bsp_release = "MCU_SDK_25.09.00+Linux_6.12.34_2.1.0"
neutron_converter_path = os.path.join(os.getenv('EIQ_BASE'), f'bin/neutron-converter/{bsp_release}/neutron-converter')

# notebook dependency check:
# eIQ environment verification
try:
    cmd = [neutron_converter_path, '--version']
    subprocess.run(cmd, check=True, stdout = subprocess.DEVNULL)
except:
    raise FileNotFoundError('Need eIQ Toolkit environment setup - see downloads/README.md')

# 
# General purpose helpers
#

# Download a list of files from urls
# returns list of local file paths
def fetch_urls(urls, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    files = []
    for url in urls:
        path = urllib.parse.urlparse(url).path
        file = os.path.basename(path)
        dest_file = os.path.join(dest_dir, file)
        print(f"downloading {url} ({dest_file})")
        # wikimedia.org requires user agent
        req = urllib.request.Request(
                  url, data=None,
                  headers= {'User-Agent' : 'download script/0.1'}
              )
        with urllib.request.urlopen(req) as resp:
            with open(dest_file, 'wb') as dest:
                content = resp.read()
                dest.write(content)
                files += [ dest_file ]
    return files

# download files and tarballs - extract archives when relevant
def fetcher(items):
    for item in items:
        url = item['url']
        dest = item['dest']
        download = fetch_urls([url], dest)[0]
        if download.endswith('.zip'):
            with zipfile.ZipFile(download) as _zip:
                print(f"extracting zip {download}")
                _zip.extractall(path=dest)
        if download.endswith('.tar.gz') or download.endswith('.tgz'):
            with tarfile.open(download) as _tar:
                print(f"extracting tar {download}")
                _tar.extractall(path=dest)

# Create list of clean (empty) directories
def make_clean_dirs(dest_dirs):
    for dest_dir in dest_dirs:
        shutil.rmtree(dest_dir, ignore_errors=True)
        Path(dest_dir).mkdir(parents=True)

# Function to load and preprocess a single image
def load_and_preprocess_image(path, input_shape):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=input_shape[3])
    image = tf.image.resize(image, [input_shape[1], input_shape[2]])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Representative dataset generator for TFLite quantization
# Use the samples if provided, else generate random ones
def representative_dataset_gen(input_shape, samples_dir=None):
    if samples_dir and os.path.exists(samples_dir):
        samples_paths = []
        for root, dirs, files in os.walk(samples_dir):
            for file in files:
                # Construct absolute file path
                absolute_path = os.path.abspath(os.path.join(root, file))
                samples_paths.append(absolute_path)
        for path in samples_paths:
            image = load_and_preprocess_image(path, input_shape)
            yield [image]
    else:
        for _ in range(100):
            yield [np.random.rand(1, input_shape[1], input_shape[2], 3).astype(np.float32)]

# Invoke TF Converter from frozen graph
def convert_frozen_graph_to_tflite(origin, dest, input_name, output_name, input_shape,
                                   quantize=False, input_type=False, output_type=False, samples_dir=False):
    input_arrays = [input_name] if isinstance(input_name, str) else input_name
    output_arrays = [output_name] if isinstance(output_name, str) else output_name

    converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
        graph_def_file=origin,
        input_arrays=input_arrays,
        output_arrays=output_arrays,
        input_shapes={input_name: input_shape}
    )
    if quantize:
        converter.post_training_quantize = True
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.inference_input_type = input_type
        converter.inference_output_type = output_type
        converter.representative_dataset = lambda: representative_dataset_gen(input_shape, samples_dir)

    tflite_model = converter.convert()

    with open(dest, 'wb') as f:
        f.write(tflite_model)
    print(f"Model converted and saved to: {dest}")

# Invoke neutron-converter with args
def neutron_converter(origin, dest, args=['--target', 'imx95']):
    cmd = [neutron_converter_path, '--input'] + [origin] + args + ['--output'] + [dest]
    subprocess.run(cmd, check=True)

In [ ]:
# Download a few jpeg picture samples for post-training quantization (for demo)
shutil.rmtree(samples_dir, ignore_errors=True)    
samples = [
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/Olea_europaea_cuspidata-africana_Cape_Town.JPG/605px-Olea_europaea_cuspidata-africana_Cape_Town.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Sport-orange-red-color-tennis-organ-491989-pxhere.jpg/640px-Sport-orange-red-color-tennis-organ-491989-pxhere.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Domesticated_goose_head%2C_Chaguaramal%2C_Venezuela.jpg/640px-Domesticated_goose_head%2C_Chaguaramal%2C_Venezuela.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/20080831-R0012506.JPG/640px-20080831-R0012506.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Pisa_Cathedral_%26_Bell_Tower_-_%22Leaning_Tower_of_Pisa%22_%289809628764%29.jpg/640px-Pisa_Cathedral_%26_Bell_Tower_-_%22Leaning_Tower_of_Pisa%22_%289809628764%29.jpg',
    ]
fetch_urls(samples, samples_dir)

# Object Detection
## Artifacts download

| models | references | 
| :- | :- |
| [ssdlite_mobilenet_v2_coco](http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz) | [TensorFlow 1 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md#mobile-models) |
| [YoloV4-tiny](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights) | [Yolo Original Implementation in Darknet](https://github.com/AlexeyAB/darknet#pre-trained-models) |

In [ ]:
from tensorflow.core.framework.graph_pb2 import GraphDef
from tensorflow.python.framework import graph_util

def load_and_prune_graph(pb_path, output_nodes):
    with tf.io.gfile.GFile(pb_path, "rb") as f:
        graph_def = GraphDef()
        graph_def.ParseFromString(f.read())

    # Keep only the subgraph needed to compute the outputs
    pruned_graph_def = graph_util.extract_sub_graph(graph_def, output_nodes)
    return pruned_graph_def


def save_graph(graph_def, output_path):
    with tf.io.gfile.GFile(output_path, "wb") as f:
        f.write(graph_def.SerializeToString())

# start from clean state
dest_dir = os.path.join(models_dir, 'object-detection')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network
model_float = 'ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz'
url_float = urllib.parse.urljoin('http://download.tensorflow.org/models/object_detection/', model_float)
packages = [
    {'url'  : url_float,
     'dest' : tmp_dir},
    {'url'  : 'https://github.com/nnsuite/testcases/raw/master/DeepLearningModels/tensorflow-lite/ssd_mobilenet_v2_coco/coco_labels_list.txt',
     'dest' : dest_dir},
    {'url'  : 'https://github.com/nnsuite/testcases/raw/master/DeepLearningModels/tensorflow-lite/ssd_mobilenet_v2_coco/box_priors.txt',
     'dest' : dest_dir},
]

fetcher(packages)

folder = re.sub('.tar.gz', '', model_float)
pb_model = os.path.join(tmp_dir, folder, 'frozen_inference_graph.pb')

model_darknet = 'yolov4-tiny.weights'
url_darknet = urllib.parse.urljoin('https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights', model_darknet)
packages = [
    {'url'  : url_darknet,
     'dest' : tmp_dir},
    {'url'  : 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-2014_2017.txt',
     'dest' : dest_dir},
]

fetcher(packages)

# Frozen graph for SSD MobileNet v2 contains Control Flow V1 ops (like tf.cond, tf.while_loop) as part of the postprocessing step.
# TFLiteConverter cannot handle complex control flow logic from older TensorFlow 1.x-style graphs unless it's rewritten using
# Control Flow V2 — possible only when having access to the original model code.
# Due to the fact that postprocessing step is not needed, raw output tensors (concat, concat_1) can be used as output nodes,
# thus graph can be pruned manually to remove everything after.
output_nodes = ['concat', 'concat_1']
pruned_graph = load_and_prune_graph(pb_model, output_nodes)
pruned_pb_model = os.path.join(tmp_dir, folder, 'pruned_model.pb')
save_graph(pruned_graph, pruned_pb_model)

## Models conversions

In [ ]:
#
# TFLite models (without box-decoding / NMS postprocessing)
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'ssdlite_mobilenet_v2_coco_no_postprocess.tflite')
convert_frozen_graph_to_tflite(
    origin=pruned_pb_model,
    dest=tflite_float,
    input_name='Preprocessor/sub',
    output_name=['concat', 'concat_1'],
    input_shape=[1, 300, 300, 3])

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'ssdlite_mobilenet_v2_coco_quant_uint8_float32_no_postprocess.tflite')
convert_frozen_graph_to_tflite(
    origin=pruned_pb_model,
    dest=tflite_quant,
    input_name='Preprocessor/sub',
    output_name=['concat', 'concat_1'],
    input_shape=[1, 300, 300, 3],
    quantize=True,
    input_type=tf.uint8,
    output_type=tf.float32)

# convert TFLite (quantized) -> TFLite (neutron)
tflite_neutron = os.path.join(dest_dir, 'ssdlite_mobilenet_v2_coco_quant_uint8_float32_no_postprocess_neutron.tflite')
neutron_converter(tflite_quant, tflite_neutron)

In [ ]:
# Convert Yolov4-tiny Darknet -> TFlite (quantized)
# this script may take several minutes to complete
!python3 {top_dir}/tasks/object-detection/yolov4-tiny_export_model.py \
--weights_path={tmp_dir} --output_path={dest_dir} --images_path={samples_dir}

#  Classification
## Artifacts download
| model | reference | 
| :- | :- |
| [mobilenet_v1_1.0_224](http://download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz) | [TensorFlow-Slim image classification model library](https://github.com/tensorflow/models/tree/master/research/slim) |

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'classification')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network
model_float = 'mobilenet_v1_1.0_224.tgz'
url_model = 'http://download.tensorflow.org/models/mobilenet_v1_2018_08_02/'
packages = [
    {'url'  : urllib.parse.urljoin(url_model, model_float),
     'dest' : tmp_dir},
    {'url'  : 'https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip',
     'dest' : tmp_dir},
]

fetcher(packages)

pb_model_float = os.path.join(tmp_dir, 'mobilenet_v1_1.0_224_frozen.pb')

labels_file = os.path.join(tmp_dir, 'labels_mobilenet_quant_v1_224.txt')
shutil.copy2(labels_file, dest_dir)


## Models conversions

In [ ]:
#
# TFLite models
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'mobilenet_v1_1.0_224.tflite')
convert_frozen_graph_to_tflite(
    origin=pb_model_float,
    dest=tflite_float,
    input_name='input',
    output_name='MobilenetV1/Predictions/Reshape_1',
    input_shape=[1, 224, 224, 3])

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'mobilenet_v1_1.0_224_quant_uint8_float32.tflite')
convert_frozen_graph_to_tflite(
    origin=pb_model_float,
    dest=tflite_quant,
    input_name='input',
    output_name='MobilenetV1/Predictions/Reshape_1',
    input_shape=[1, 224, 224, 3],
    quantize=True,
    input_type=tf.uint8,
    output_type=tf.float32,
    samples_dir=samples_dir)

# convert TFLite (quantized) -> TFLite (neutron)
tflite_neutron = os.path.join(dest_dir, 'mobilenet_v1_1.0_224_quant_uint8_float32_neutron.tflite')
neutron_converter(tflite_quant, tflite_neutron)

# Semantic Segmentation
## Artifacts download
| model | references | 
| :- | :- |
| [deeplabv3_mnv2_dm05](http://download.tensorflow.org/models/deeplabv3_mnv2_dm05_pascal_trainaug_2018_10_01.tar.gz) | [TensorFlow DeepLab Model Zoo](https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/model_zoo.md) <br> [Quantize DeepLab model for faster on-device inference](https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/quantize.md) |

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'semantic-segmentation')
images_dir = os.path.join(media_dir, 'pascal_voc_2012_images')
make_clean_dirs([tmp_dir, dest_dir, images_dir])

# models and metadata to be downloaded from network
model_float = 'deeplabv3_mnv2_dm05_pascal_trainaug_2018_10_01.tar.gz'
url_model = 'http://download.tensorflow.org/models/'
packages = [
    {'url'  : urllib.parse.urljoin(url_model, model_float),
     'dest' : tmp_dir},
]

fetcher(packages)

pb_model_float = os.path.join(tmp_dir, 'deeplabv3_mnv2_dm05_pascal_trainaug','frozen_inference_graph.pb')

# fetch and rename a few images relevant to pascal voc 2012 data set
images = [
    'https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Plectranthus_verticillatus_in_a_ceramic_pot.jpg/640px-Plectranthus_verticillatus_in_a_ceramic_pot.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Maceta.jpg/566px-Maceta.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Stray_Cat%2C_Nafplio.jpg/800px-Stray_Cat%2C_Nafplio.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Feral_cat_7.jpg/800px-Feral_cat_7.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/0/0a/Senior-lady-dog-walker.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Texel_ewe_and_three_lambs.jpg/800px-Texel_ewe_and_three_lambs.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Cow_farm_in_Bemmel%2C_Lingewaard.jpg/800px-Cow_farm_in_Bemmel%2C_Lingewaard.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Urban_cycling_III.jpg/800px-Urban_cycling_III.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2e/LomondRoadsTTRider.jpg/467px-LomondRoadsTTRider.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Interceptor_of_Bangalore_Traffic_Police.jpg/640px-Interceptor_of_Bangalore_Traffic_Police.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Ranger_clearing_traffic_at_a_bison_jam_in_Lamar_Valley_%2848fe8fe3-c1e3-4828-acec-496eaef43503%29.jpg/800px-Ranger_clearing_traffic_at_a_bison_jam_in_Lamar_Valley_%2848fe8fe3-c1e3-4828-acec-496eaef43503%29.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c7/Finnair.a320-200.oh-lxf.arp.jpg/800px-Finnair.a320-200.oh-lxf.arp.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/G-BGMP_Reims_F172_%40Cotswold_Airport%2C_July_2005.jpg/800px-G-BGMP_Reims_F172_%40Cotswold_Airport%2C_July_2005.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/JNR_14_series_sleeper_Blue_Train_Hayabusa.jpg/640px-JNR_14_series_sleeper_Blue_Train_Hayabusa.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/7122_series_train_%2812%29.JPG/640px-7122_series_train_%2812%29.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Blue_fishing_boat_harbour_Eretria_Euboea_Greece.jpg/611px-Blue_fishing_boat_harbour_Eretria_Euboea_Greece.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/KYOEI_boat_in_Kisarazu_port_3.jpg/640px-KYOEI_boat_in_Kisarazu_port_3.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Jarritos_glass_bottle_%28Mexico%29.jpg/450px-Jarritos_glass_bottle_%28Mexico%29.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Andechser_3_beers.JPG/798px-Andechser_3_beers.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Ski_Citrus_Soda_%2825395355114%29.jpg/640px-Ski_Citrus_Soda_%2825395355114%29.jpg',
]
files = fetch_urls(images, tmp_dir)
count = 0
# rename files name as <image%04d.jpg>
for src in files:
    dest = os.path.join(images_dir, f'image{count:04}.jpg')
    count += 1
    shutil.copy2(src, dest)


## Models conversions

In [ ]:
#
# TFLite models
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'deeplabv3_mnv2_dm05_pascal.tflite')
convert_frozen_graph_to_tflite(
    origin=pb_model_float,
    dest=tflite_float,
    input_name='MobilenetV2/MobilenetV2/input',
    output_name='ResizeBilinear_2',
    input_shape=[1, 513, 513, 3])

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'deeplabv3_mnv2_dm05_pascal_quant_uint8_float32.tflite')
convert_frozen_graph_to_tflite(
    origin=pb_model_float,
    dest=tflite_quant,
    input_name='MobilenetV2/MobilenetV2/input',
    output_name='ResizeBilinear_2',
    input_shape=[1, 513, 513, 3],
    quantize=True,
    input_type=tf.uint8,
    output_type=tf.float32,
    samples_dir=samples_dir)

# Pose Estimation
## Artifacts download
| model | references | 
| :- | :- |
| [movenet/singlepose/lightning TF2 SavedModel](https://tfhub.dev/google/movenet/singlepose/lightning/4) | [TensorFlow Hub Image Pose Detection](https://tfhub.dev/s?module-type=image-pose-detection) <br> [TensorFlow Pose](https://www.tensorflow.org/lite/examples/pose_estimation/overview) |

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'pose-estimation')
make_clean_dirs([tmp_dir, dest_dir])
movies_dir = os.path.join(media_dir, 'movies')

# models and metadata to be downloaded from network

# Download movie
url_movie = 'https://upload.wikimedia.org/wikipedia/commons/transcoded/1/17/Conditioning_Drill_1-_Power_Jump.webm/Conditioning_Drill_1-_Power_Jump.webm.480p.vp9.webm'
packages = [{'url': url_movie, 'dest': movies_dir},]
fetcher(packages)

# quantized model to be downloaded from network

url_models = 'https://github.com/NXP/nxp-vision-model-zoo/releases/download/v1.1/nxp_eiq_vision_model_zoo_v1.1.tgz'
packages = [{'url' :url_models, 'dest': tmp_dir},]
fetcher(packages)

models_tmp_dir = os.path.join(tmp_dir, 'tasks')

!cp {models_tmp_dir}/pose-estimation/movenet/movenet.tflite \
{dest_dir}/movenet_quant.tflite

# Face Processing
## Artifacts download
| models | references | 
| :- | :- |
| [FaceNet512](https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5) | [Deepface repository](https://github.com/serengil/deepface/tree/master/deepface/basemodels) |
| [Ultraface-slim](https://github.com/Linzaer/Ultra-Light-Fast-Generic-Face-Detector-1MB/blob/master/models/pretrained/version-slim-320.pth) | [Ultra-Light-Fast-Generic-Face-Detector repository](https://github.com/Linzaer/Ultra-Light-Fast-Generic-Face-Detector-1MB) |
|[Deepface-emotion](https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5) | [Deepface repository](https://github.com/serengil/deepface/blob/master/deepface/extendedmodels/Emotion.py) |

These models are downloaded from [NXP vision model zoo](https://github.com/NXP/nxp-vision-model-zoo) where they have been quantized

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'face-processing')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network

url_models = 'https://github.com/NXP/nxp-vision-model-zoo/releases/download/v1.0/nxp_eiq_vision_model_zoo_v1.0.tgz'
packages = [{'url' :url_models, 'dest': tmp_dir},]
fetcher(packages)

models_tmp_dir = os.path.join(tmp_dir, 'tasks')

!cp {models_tmp_dir}/object-detection/ultraface-slim/ultraface_slim_uint8_float32.tflite \
{models_tmp_dir}/face-recognition/facenet512/facenet512_uint8.tflite \
{models_tmp_dir}/classification/deepface-emotion/emotion_uint8_float32.tflite \
{dest_dir}/

## Model conversions

In [ ]:
# convert TFLite (quantized) -> TFLite (neutron)
tflite_quant = os.path.join(models_tmp_dir, 'object-detection/ultraface-slim/ultraface_slim_uint8_float32.tflite')
tflite_neutron = os.path.join(dest_dir, 'ultraface_slim_uint8_float32_neutron.tflite')
neutron_converter(tflite_quant, tflite_neutron)

## Monocular Depth Estimation
## Artifacts download
| models | references | 
| :- | :- |
| Midasv2 | [Midas repository](https://www.kaggle.com/models/intel/midas/tfLite/v2-1-small-lite/1) |

The model is downloaded from kaggle, its performances and quantization process are available on [NXP vision model zoo](https://github.com/NXP/nxp-vision-model-zoo)

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'monocular-depth-estimation')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network

model_float = os.path.join(dest_dir, '')
url_models = 'https://www.kaggle.com/api/v1/models/intel/midas/tfLite/v2-1-small-lite/1/download/1.tflite'
packages = [{'url' :url_models, 'dest': dest_dir},]
fetcher(packages)

model_float = os.path.join(dest_dir, '1.tflite')
renamed_model_float = os.path.join(dest_dir, 'midas_2_1_small_float32.tflite')
os.rename(model_float, renamed_model_float)

!cp {renamed_model_float} {tmp_dir}/midas_2_1_small_float32.tflite

## Model conversions

In [ ]:
os.chdir(tmp_dir)

# convert TFLite -> TF for quantization process
req_dir = os.path.join(top_dir, 'tasks/monocular-depth-estimation/requirements.txt')
script_dir = os.path.join(top_dir, 'tasks/monocular-depth-estimation/export_midas-v2_to_TensorFlow.sh')
os.system('chmod a+x ' + script_dir)
os.system('bash ' + script_dir + ' ' + req_dir)

# convert TF -> TFLite (quantized)
pb_model = os.path.join(tmp_dir, 'model_float32.pb')
tflite_quant = os.path.join(dest_dir, 'midas_2_1_small_int8_quant.tflite')
convert_frozen_graph_to_tflite(
    origin=pb_model,
    dest=tflite_quant,
    input_name='Const',
    output_name='midas_net_custom/sequential/re_lu_9/Relu',
    input_shape=[1, 256, 256, 3],
    quantize=True,
    input_type=tf.uint8,
    output_type=tf.float32)

# Postamble / cleanup

In [ ]:
# Remove temporary directories
shutil.rmtree(tmp_dir, ignore_errors=True)
shutil.rmtree(samples_dir, ignore_errors=True)